<a href="https://colab.research.google.com/github/DebuggerAu/Demos/blob/master/Reinforcement_Learning_for_Stock_Market_Trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Stage 1: Installing dependencies and environment setup


In [ ]:
!pip install tensorflow-gpu==2.0.0rc0

     |████████████████████████████████| 380.5MB 85kB/s 
     |████████████████████████████████| 4.3MB 37.9MB/s 
     |████████████████████████████████| 501kB 38.5MB/s 


In [22]:
!pip install pandas-datareader
!pip install yfinance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.2/948.2 kB 15.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.7/104.7 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.4/117.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 8.8 MB/s eta 0:00:00
  Created wheel for peewee: filename=peewee-3.17.8-cp310-cp310-linux_x86_64.whl size=843071 sha256=6aad5fc84090b81228ef667826da79836a4fe3d10e0a7681b1e04172f1faa333
  Stored in directory: /root/.cache/pip/wheels/75/79/e5/8838db0594cc6c587142fd2563356392ade6255c5930411069
Successfully built peewee


## Stage 2: Importing project dependencies

In [23]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader
import yfinance as yf
from tqdm import tqdm_notebook, tqdm
from collections import deque

In [24]:
tf.__version__

'2.15.0'

## Stage 3: Building the AI Trader network

In [43]:
class AI_Trader():

  def __init__(self, state_size, action_space=3, model_name="AITrader"): #Stay, Buy, Sell

    self.state_size = state_size
    self.action_space = action_space
    self.memory = deque(maxlen=2000)
    self.inventory = []
    self.model_name = model_name

    # Define hyperparamaters
    self.gamma = 0.95
    self.epsilon = 1.0
    self.epsilon_final = 0.01
    self.epsilon_decay = 0.995

    # Call a function  to build a model trought this class constructor
    # More parameters could be ustilized to programaticaly define network size (layers and neurons)
    self.model = self.model_builder()


  def model_builder(self):

    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units=32, activation='relu', input_dim=self.state_size))
    model.add(tf.keras.layers.Dense(units=64, activation='relu'))
    model.add(tf.keras.layers.Dense(units=128, activation='relu'))
    model.add(tf.keras.layers.Dense(units=self.action_space, activation='linear'))
    model.compile(loss='mse', optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001))
    return model




  # Trade function that takes state as an input and returns an action
  # to perform in perticular state
  def trade(self, state):

    # Should we perform a renadom generated action or action defined in model?

    # If value from our random generator is smaller or equal to our epsilon
    #     then we will retun a random action from action_space [0-3)
    if random.random() <= self.epsilon:
      return random.randrange(self.action_space)

    # If our random is greater than epsilon then we will use model to perform action
    actions = self.model.predict(state)
    # return only a one number defining an action (#Stay - 0 , Buy - 1, Sell - 2)
    #    that has maximum probability
    return np.argmax(actions[0])



  def batch_train(self, batch_size):

    batch = []

    # Iterrate in momory, we do not want to randolmy select data as we are dealing with
    #    time constraint data. We will always sample from the end of memory size of bath
    for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
      # insert data from memory to batch
      batch.append(self.memory[i])


    # Iterate trought batch of data and train the model for each sample from batch
    # Order of variables in for loop is important
    for state, action, reward, next_state, done in batch:
      # Reward if agent is in terminal state
      reward = reward
      # Check that agent is not in terminal state
      # If not in terminal state calculate reward for actions that could be played
      if not done:
        # Discounted total reward:
        reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
      # Target variable that is predicted by the model (action)
      target = self.model.predict(state)
      target[0][action] = reward

      self.model.fit(state, target, epochs=1, verbose=0)

    # We will decrease epsilon parameter that is 1 as defined in __init__  so
    #    so we can stop performing random actions at some point
    if self.epsilon > self.epsilon_final:
      self.epsilon *= self.epsilon_decay

## Stage 4: Dataset preprocessing

### Defining helper functions

#### Sigmoid

In [26]:
# Usually used at the end of a network for binary classifictation
# It changes range of input to scale of [0,1]
# So we can normalize input data for comparision day by day if they are on different scale
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

#### Price format function

In [27]:
def stocks_price_format(n):
  if n < 0:
    return "- $ {0:2f}".format(abs(n))
  else:
    return "$ {0:2f}".format(abs(n))

#### Dataset loader

In [29]:
# Check the data gathered with pandas data_reader:
dataset = yf.download('SPY', start = '2012-01-01', end='2025-01-01')

#data_reader.DataReader(name="AAPL", data_source="google")
print("Data set top rows:", "\n" ,dataset.head())

print("Test some cutting with pandas")
print("Start date: ", str(dataset.index[0]).split()[0])
print("End date: ", str(dataset.index[-1]).split()[0])

[*********************100%***********************]  1 of 1 completed

Data set top rows: 
 Price            Close        High         Low        Open     Volume
Ticker             SPY         SPY         SPY         SPY        SPY
Date                                                                 
2012-01-03  100.752831  101.448226  100.697515  100.958289  193697900
2012-01-04  100.910919  100.997844  100.128604  100.515810  127186500
2012-01-05  101.179565  101.329709   99.907319  100.365647  173895000
2012-01-06  100.918777  101.321790  100.586887  101.305983  148050000
2012-01-09  101.163780  101.290206  100.681746  101.147972   99530200
Test some cutting with pandas
Start date:  2012-01-03
End date:  2024-12-31


In [30]:
def dataset_loader(stock_name, web_data_source="yahoo"):

  #Use pandas data reader for reading stock data from warious sources like "yahoo", "google"
  dataset = data_reader.DataReader(name=stock_name, data_source="yahoo")

  # Get start and end time to variables from dataset
  start_date = str(dataset.index[0]).split()[0]
  end_date = str(dataset.index[-1]).split()[0]

  # Model will use "Close" column for training
  close = dataset['Close']

  return close

### State creator

In [48]:
# Data -> dataset to predict from, gathered by data:loader()
# Timestep -> Day in the dataset that we want to predict for [0:datalength]
# window_suze -> how many days in past we want to use to predict current status[1:datalength]
#         Try different setup to see what creates best fit
def state_creator(data, timestep, window_size):

  # starting day of our state
  starting_id = timestep - window_size + 1

  if starting_id >= 0:
    windowed_data = data[starting_id:timestep+1]
  else:
    # Replicate member (data[0]) needed times
    windowed_data = - starting_id * [data[0]] + list(data[0:timestep+1])

  state = []
  # Iterate trough whole windowed_data minus current state (-1)
  for i in range(window_size - 1):
    # Normalize the difference from current day and the next day
    # Because the prices can be very different and we want them on same scale
    state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))

  return np.array([state])

### Loading a dataset

In [33]:
# stage data for Apple
stock_name = "AAPL"
data = dataset = yf.download(stock_name , start = '2012-01-01', end='2025-01-01')

[*********************100%***********************]  1 of 1 completed


## Stage 5: Training the AI Trader

### Setting hyper parameters

In [40]:
window_size = 10
episodes = 1000 # same as epoch
epochs = 50
learning_rate = 0.01
decay_rate = learning_rate / epochs
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=learning_rate, decay=decay_rate)

batch_size = 32
data_samples = len(data) - 1 # discard last value, that we will predict on

### Defining the Trader model

In [45]:
trader = AI_Trader(window_size)

In [46]:
trader.model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 32)                352       
                                                                 
 dense_17 (Dense)            (None, 64)                2112      
                                                                 
 dense_18 (Dense)            (None, 128)               8320      
                                                                 
 dense_19 (Dense)            (None, 3)                 387       
                                                                 
Total params: 11171 (43.64 KB)
Trainable params: 11171 (43.64 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Training loop

In [49]:
for episode in range(1, episodes + 1):

  # To keep track of training process
  # .format populates {} with variables in .format(x,y)
  print("Episode: {}/{}".format(episode, episodes))

  # Create state
  # second parameter is timestep = 0
  state = state_creator(data, 0, window_size + 1)

  total_profit = 0
  # Empty inventory before starting episode
  trader.inventory = []

  # One timestep is one day so number of timesteps we have represent data we have
  # tqdm is used for visualization
  for t in tqdm(range(data_samples)):

    # First we will access action that is going to be taken by model
    action = trader.trade(state)

    # Use action to get to next state(t+)
    next_state = state_creator(data, t+1, window_size + 1)
    # As we did not calculate anything up to this point reward is 0
    reward = 0

    if action == 1: #Buying
      # Put buyed stock to inventory to trade with
      trader.inventory.append(data[t])
      print("AI Trader bought: ", stocks_price_format(data[t]))

    # To sell we need to have something in inventory
    elif action == 2 and len(trader.inventory) > 0: #Selling
      # Check buy price, pop removes first value from list
      buy_price = trader.inventory.pop(0)

      # If we gain money (current price - buy price) we have reward
      #    if we lost money then reward is 0
      reward = max(data[t] - buy_price, 0)
      total_profit += data[t] - buy_price
      print("AI Trader sold: ", stocks_price_format(data[t]), " Profit: " + stocks_price_format(data[t] - buy_price) )

    # if t is last sample in our dateset we are done
    #     we do not have any steps to perform in current episode
    if t == data_samples - 1:
      done = True
    else:
      done = False

    # Append all data to trader-agent memory, experience buffer
    trader.memory.append((state, action, reward, next_state, done))

    # change state to next state, so we are done with an episode
    state = next_state

    if done:
      print("########################")
      print("TOTAL PROFIT: {}".format(total_profit))
      print("########################")

    # Chekc if we have more information in our memory than batch size
    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)

  # Save the model every 10 episodes
  if episode % 10 == 0:
    trader.model.save("ai_trader_{}.h5".format(episode))


Episode: 1/1000


KeyError: 0